<a href="https://colab.research.google.com/github/kode-git/Copernicus-river-discharges/blob/main/Initial_Exploratory_Spatial_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Exploratory Spatial Data Analysis

## Library Dependencies

In [18]:
!pip install xarray 
!pip install netCDF4 dask bottleneck
!pip install pandas
!pip install geopandas
!pip install cdsapi

In [65]:
# collections libraries
import netCDF4 as nc4
from netCDF4 import Dataset
import xarray as xr
import numpy as np

# file management
from glob import glob

# utilities 
from datetime import datetime as dt

# colab
from google.colab import drive

In [20]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Samples Loading

In [24]:
!chmod 777 /content/drive/MyDrive/Datasets/cs3-copernicus-datasets/cps3-copernicus-dataset-tpi-rdh/rdh/rdh_2022_2021.nc

In [38]:
nc = Dataset("/content/drive/MyDrive/Datasets/cs3-copernicus-datasets/cps3-copernicus-dataset-tpi-rdh/rdh/rdh_2022_2021.nc", "r", "NETCDF4" )
for dim in nc.dimensions.values():
     print(dim, dim.isunlimited())

<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 950 False
<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 1000 False
<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 498 False


We have 3 dimensions on data related to the coordinates (x,y) and the time from the time series of the dataset. Let's check the size for each dimension.

In [40]:
for name in nc.dimensions.keys():
    dim = nc.variables[name]
    print(name, dim.dtype, dim.size)

y float64 950
x float64 1000
time int64 498


## Attributes Visualization

In [46]:
def print_ncattr(fid, key):
    """
        Prints the NetCDF file attributes for a given key

        Parameters: 
            * fid:  netCDF file identifier
            * key:  unicode (a valid netCDF4.Dataset.variables key)
    """
    try:
        print('{}  -->'.format(key))
        print("\t {:>15}: {}".format("type", fid.variables[key].dtype))
        for attr in fid.variables[key].ncattrs():
            print('\t {:>15}: {}'.format(attr, fid.variables[key].getncattr(attr)))
    except KeyError:
        print("\t WARNING: {} does not contain variable attributes".format(key))

In [47]:
print(print_ncattr.__doc__)


        Prints the NetCDF file attributes for a given key

        Parameters: 
            * fid:  netCDF file identifier
            * key:  unicode (a valid netCDF4.Dataset.variables key)
    


In [48]:
for name in nc.variables.keys():
    print_ncattr(nc, name)

y  -->
	            type: float64
	      _FillValue: nan
	           units: Meter
	       long_name: Y coordinate of projection
	   standard_name: projection_Y_coordinate
x  -->
	            type: float64
	      _FillValue: nan
	           units: Meter
	       long_name: x coordinate of projection
	   standard_name: projection_x_coordinate
time  -->
	            type: int64
	       long_name: initial time of forecast
	   standard_name: forecast_reference_time
	           units: seconds since 1970-01-01
	        calendar: proleptic_gregorian
step  -->
	            type: float64
	      _FillValue: nan
	       long_name: time since forecast_reference_time
	   standard_name: forecast_period
	           units: hours
surface  -->
	            type: float64
	      _FillValue: nan
	       long_name: original GRIB coordinate for key: level(surface)
	           units: 1
latitude  -->
	            type: float32
	      _FillValue: nan
	    grid_mapping: lambert_azimuthal_equal_area
	       long_na

## Variables Information

In [69]:
!pip3 uninstall matplotlib -y
!apt-get install python3-matplotlib

Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-matplotlib-data python3-cycler python3-dateutil python3-numpy
  python3-olefile python3-pil python3-pyparsing python3-six python3-tz
  ttf-bitstream-vera
Suggested packages:
  apache2 | lighttpd | httpd libjs-jquery-ui-docs python-cycler-doc dvipng
  gir1.2-gtk-3.0 ghostscript inkscape ipython3 python-matplotlib-doc
  python3-cairocffi python3-gi-cairo python3-gobject python3-nose
  python3-pyqt4 python3-scipy python3-sip python3-tornado texlive-extra-utils
  texlive-latex-extra ttf-staypuft python-numpy-doc python3-numpy

In [70]:
lon = nc.variables['longitude'][:]
lat = nc.variables['latitude'][:]
time = nc.variables['time'][:]
dis = nc.variables['dis06'][:]
x = nc.variables['x'][:]
y = nc.variables['y'][:]